# Graph Node

In [ ]:
import torch
from torch import nn

In [7]:
layer = Node2NodeLayer(2, 5)

In [8]:
y = layer(x, adj_matrix, 'sum')

In [10]:
# [1, 3, 2]

x = torch.randn((1, 3, 3))
e = torch.randn((1, 3, 1))

adj_matrix = torch.Tensor([
    [
        [0., 1., 1.],
        [0., 0., 0.],
        [0., 0., 0.]
    ]
])

In [11]:
layer = Node2EdgeLayer(3, 1, 5)

In [12]:
eps = layer(x, e, adj_matrix)

In [13]:
eps

tensor([[[0.4753, 0.4541, 0.4873, 0.5478, 0.3932],
         [0.3953, 0.6331, 0.4715, 0.3416, 0.2130],
         [0.4423, 0.5090, 0.5684, 0.2750, 0.3456]]],
       grad_fn=<SigmoidBackward0>)

In [17]:
x = torch.randn((1, 3, 3))
e = torch.randn((1, 3, 1))

adj_matrix = torch.Tensor([
    [
        [0., 1., 1.],
        [0., 0., 0.],
        [0., 0., 0.]
    ]
])

In [18]:
a_test = Edge2NodeLayer(1,3,6)

In [20]:
y = a_test(x,e,adj_matrix)
y

tensor([[[0.5343, 0.4041, 0.5649, 0.5820, 0.6596, 0.4498],
         [0.4732, 0.6386, 0.7799, 0.5576, 0.6592, 0.4158],
         [0.4907, 0.4598, 0.5973, 0.5634, 0.5139, 0.3807]]],
       grad_fn=<SigmoidBackward0>)

# Encoder-Decoder

In [2]:
from enc_and_dec import Encoder,Decoder
from simple_GAN import Discriminator
import option_parser
import torch

d:\anaconda3\envs\keraal\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
parser = option_parser.get_parser()
args, unknown = parser.parse_known_args()

In [4]:
topo = (0, 0, 1, 2, 3, 0, 5, 6, 7, 0, 9, 10, 11, 12, 13, 12, 15, 16, 17, 12, 19, 20, 21)
data = torch.rand((256, 64, 23, 3))
offset = torch.rand((256, 64, 23, 1))

In [5]:
#test encoder, decoder
enc = Encoder(args, topo)

z = enc(data, offset)
print(z.shape)
dec = Decoder(args,topo)
offset2 = torch.rand((8,23,1))
reconstruct = dec(z, offset2)
print(reconstruct.shape)

RuntimeError: Sizes of tensors must match except in dimension 2. Expected size 3 but got size 1 for tensor number 1 in the list.

In [6]:
dis = Discriminator(args,topo)
data = torch.rand((64, 23, 3))
offset = torch.rand((64, 23, 1))

new_input = torch.concat([data,offset],dim = 2)
res = dis(new_input)

# Network

In [1]:
from torch.utils.data.dataloader import DataLoader
from data import create_dataset, get_character_names
import option_parser
from models.architecture import GAN_model
import torch
args = option_parser.get_args()
characters = get_character_names(args)
dataset = create_dataset(args, characters)
print(len(dataset))
data_loader = DataLoader(dataset, batch_size = args.batch_size, shuffle=True, num_workers=0)
model = GAN_model(args, dataset)
for step, motions in enumerate(data_loader):
    print("step:", step)
    for motion in motions:
        # print(idx.shape)   
        model.set_input(motion)
        model.optimize_parameters()

d:\anaconda3\envs\keraal\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


load from file ./datasets/npy/Abe.npy
Window count: 219, total frame: 10960
load from file ./datasets/npy/Adam.npy
Window count: 219, total frame: 10741
load from file ./datasets/npy/Amy.npy
Window count: 170, total frame: 8790
170
step: 0
update
update
update
step: 1
update
update
update
step: 2
update
update
update
step: 3
update
update
update
step: 4
update
update
update
step: 5
update
update
update


In [6]:
a = torch.rand(23).unsqueeze(-1).unsqueeze(0)
print(a.shape)

torch.Size([1, 23, 1])


In [3]:
import torch.nn as nn
m = nn.Linear(3, 2)
input = torch.randn(128, 64, 23, 3)
output = m(input)
print(output.size())

torch.Size([128, 64, 23, 2])


In [1]:
from models.skeleton_gnn import Node2NodeLayer, Node2EdgeLayer, Edge2NodeLayer
import torch
def calculate_adj_matrix(topology):
    adj = torch.zeros(len(topology), len(topology))
    for i,j in enumerate(topology):
        if i == 0: #the root not in
            continue
        adj[j,i] = 1
    return adj

topo111 = [0, 0, 1, 2, 3, 0, 5, 6, 7, 0, 9, 10, 11, 12, 13, 12, 15, 16, 17, 12, 19, 20, 21]
input_node = torch.rand((128, 32, 23, 3))
input_edge = torch.rand((128, 32, 23, 1))
batch_size = 128
frame = 32
length = 23
adj = calculate_adj_matrix(topo111)
adj_matrix = torch.cat(batch_size*frame*[adj]).reshape((batch_size, frame, length, length))
layer2 = Edge2NodeLayer(1,3,3)
output = layer2(input_node, input_edge, adj_matrix)


d:\anaconda3\envs\keraal\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
output.shape

torch.Size([128, 32, 23, 3])